In [4]:
import threading
import time

import pandas as pd

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.trade_app import TradeApp
import lib.helper as helper

In [5]:
app = TradeApp()


In [6]:
"""Function that run the entire trading app """
app = TradeApp()
app.connect(host='127.0.0.1', port=7497,
                clientId=23)  # port 4002 for ib gateway paper trading/7497 for TWS paper trading
con_thread = threading.Thread(target=app.run, daemon=True)
con_thread.start()

tickers = ["FB", "AMZN", "INTC", "MSFT", "AAPL", "GOOG", "CSCO", "CMCSA", "ADBE", "NVDA",
               "NFLX", "PYPL", "AMGN", "AVGO", "TXN", "CHTR", "QCOM", "GILD", "FISV", "BKNG",
               "INTU", "ADP", "CME", "TMUS", "MU"]

capital = 1000

app.data = {}
app.pos_df = pd.DataFrame(columns=['Account', 'Symbol', 'SecType',
                                       'Currency', 'Position', 'Avg cost'])
app.order_df = pd.DataFrame(columns=['PermId', 'ClientId', 'OrderId',
                                         'Account', 'Symbol', 'SecType',
                                         'Exchange', 'Action', 'OrderType',
                                         'TotalQty', 'CashQty', 'LmtPrice',
                                         'AuxPrice', 'Status'])
app.reqPositions()
time.sleep(2)
pos_df = app.pos_df
pos_df.drop_duplicates(inplace=True, ignore_index=True)  # position callback tends to give duplicate values
app.reqOpenOrders()
time.sleep(2)
ord_df = app.order_df

def histData(req_num,contract,duration,candle_size):
    """extracts historical data"""
    app.reqHistoricalData(reqId=req_num, 
                          contract=contract,
                          endDateTime='',
                          durationStr=duration,
                          barSizeSetting=candle_size,
                          whatToShow='ADJUSTED_LAST',
                          useRTH=1,
                          formatDate=1,
                          keepUpToDate=0,
                          chartOptions=[])	 # EClient function to request contract details


for ticker in tickers:
    print("starting passthrough for.....",ticker)
    histData(tickers.index(ticker),helper.contract(ticker),'1 M', '15 mins')
    time.sleep(5)
    df = dataDataframe(app,tickers,ticker)
    df["stoch"] = stochOscltr(df)
    df["macd"] = MACD(df)["MACD"]
    df["signal"] = MACD(df)["Signal"]
    df["atr"] = atr(df,60)
    df.dropna(inplace=True)
    quantity = int(capital/df["Close"][-1])
    if quantity == 0:
        continue
    if len(pos_df.columns)==0:
        if df["macd"][-1]> df["signal"][-1] and \
            df["stoch"][-1]> 30 and \
            df["stoch"][-1] > df["stoch"][-2]:
                app.reqIds(-1)
                time.sleep(2)
                order_id = app.nextValidOrderId
                app.placeOrder(order_id,usTechStk(ticker),marketOrder("BUY",quantity))
                time.sleep(5)
                try:
                    pos_df = app.pos_df
                    time.sleep(5)
                    sl_q = pos_df[pos_df["Symbol"]==ticker]["Position"].sort_values(ascending=True).values[-1]
                    app.placeOrder(order_id+1,usTechStk(ticker),stopOrder("SELL",sl_q,round(df["Close"][-1]-df["atr"][-1],1)))
                except Exception as e:
                    print(e, "no fill for {}".format(ticker))
          
    elif len(pos_df.columns)!=0 and ticker not in pos_df["Symbol"].tolist():
        if df["macd"][-1]> df["signal"][-1] and \
            df["stoch"][-1]> 30 and \
            df["stoch"][-1] > df["stoch"][-2]:
                app.reqIds(-1)
                time.sleep(2)
                order_id = app.nextValidOrderId
                app.placeOrder(order_id,usTechStk(ticker),marketOrder("BUY",quantity))
                time.sleep(5)
                try:
                    pos_df = app.pos_df
                    time.sleep(5)
                    sl_q = pos_df[pos_df["Symbol"]==ticker]["Position"].sort_values(ascending=True).values[-1]
                    app.placeOrder(order_id+1,usTechStk(ticker),stopOrder("SELL",sl_q,round(df["Close"][-1]-df["atr"][-1],1)))
                except Exception as e:
                    print(e, "no fill for {}".format(ticker))
                   
    elif len(pos_df.columns)!=0 and ticker in pos_df["Symbol"].tolist():
        if pos_df[pos_df["Symbol"]==ticker]["Position"].sort_values(ascending=True).values[-1] == 0:
            if df["macd"][-1]> df["signal"][-1] and \
               df["stoch"][-1]> 30 and \
               df["stoch"][-1] > df["stoch"][-2]:
                app.reqIds(-1)
                time.sleep(2)
                order_id = app.nextValidOrderId
                app.placeOrder(order_id,usTechStk(ticker),marketOrder("BUY",quantity))
                time.sleep(5)
                try:
                    pos_df = app.pos_df
                    time.sleep(5)
                    sl_q = pos_df[pos_df["Symbol"]==ticker]["Position"].sort_values(ascending=True).values[-1]
                    app.placeOrder(order_id+1,usTechStk(ticker),stopOrder("SELL",sl_q,round(df["Close"][-1]-df["atr"][-1],1)))
                except Exception as e:
                    print(e, "no fill for {}".format(ticker))
        elif pos_df[pos_df["Symbol"]==ticker]["Position"].sort_values(ascending=True).values[-1] > 0:
            try:
                ord_id = ord_df[ord_df["Symbol"]==ticker]["OrderId"].sort_values(ascending=True).values[-1]
                old_quantity = pos_df[pos_df["Symbol"]==ticker]["Position"].sort_values(ascending=True).values[-1]
                app.cancelOrder(ord_id)
                app.reqIds(-1)
                time.sleep(2)
                order_id = app.nextValidOrderId
                app.placeOrder(order_id,usTechStk(ticker),stopOrder("SELL",old_quantity,round(df["Close"][-1]-df["atr"][-1],1)))
            except Exception as e:
                print(ticker,e)

ERROR -1 504 Not connected
ERROR 0 504 Not connected


starting passthrough for..... FB


NameError: name 'dataDataframe' is not defined

In [ ]:
pos_df